In [22]:
import os

# IMPORTANT: move working directory to project root
os.chdir("/Users/sakshigandhi/Desktop/dot_stolen_content_project")

print("New CWD:", os.getcwd())
print("Files:", os.listdir())
print("Data folder:", os.listdir("data"))

New CWD: /Users/sakshigandhi/Desktop/dot_stolen_content_project
Files: ['.DS_Store', 'new_venv', 'src:', '.venv', 'sql:', 'notebooks:', 'data']
Data folder: ['.DS_Store', 'posts.csv', 'users.csv', 'harm_by_country_summary.csv', 'policy_metrics_summary.csv', 'feed_impressions.csv', 'posts_with_predictions.csv']


# Policy Simulation for Stolen Content on DOT

In this notebook I use the synthetic DOT dataset to simulate product policies
for handling stolen posts. I start from the feed level impressions and the
model prediction `pred_is_stolen` from the previous step and then ask:

1) What fraction of feed impressions currently go to stolen posts  
2) What happens to harm and engagement if we down rank or hide stolen content  
3) How these trade offs look for different creator segments


In [23]:
import pandas as pd
import numpy as np

# Load tables from the /data folder in the project root
posts = pd.read_csv("data/posts_with_predictions.csv")
impr  = pd.read_csv("data/feed_impressions.csv")
users = pd.read_csv("data/users.csv")

posts.head(), impr.head(), users.head()


(   post_id  author_id  group_id  is_original  is_stolen  created_at  \
 0        1       2986         1         True      False  2024-08-04   
 1        2        559         2         True      False  2024-07-17   
 2        3       1170         3         True      False  2024-07-24   
 3        4         21         4         True      False  2024-07-26   
 4        5        130         5         True      False  2024-07-05   
 
   media_type                                          text  like_count  \
 0      image   Content group 1 original post about topic 9          37   
 1      image  Content group 2 original post about topic 18           2   
 2      video  Content group 3 original post about topic 23          28   
 3      video  Content group 4 original post about topic 34          20   
 4      video   Content group 5 original post about topic 4           9   
 
    comment_count  share_count                               post_text_clean  \
 0              9            1   c

## Step 1  Build an impression level dataset

Here I join feed impressions with post level labels and user attributes.
This lets me evaluate how different policies impact both engagement and creator exposure.


In [24]:
import pandas as pd
import numpy as np

# we already ran the os.chdir cell and loaded these
posts = pd.read_csv("data/posts_with_predictions.csv")
impr  = pd.read_csv("data/feed_impressions.csv")
users = pd.read_csv("data/users.csv")

# keep just the fields we need
posts_small = posts[[
    "post_id",
    "author_id",
    "is_stolen",
    "pred_is_stolen"
]]

users_small = users[[
    "user_id",
    "country",
    "creator_type"    # if you created this in step 1, otherwise drop this column
]]

# impression level join
impr_posts = (
    impr
    .merge(posts_small, on="post_id", how="left")
    .merge(users_small, left_on="author_id", right_on="user_id", how="left")
)

impr_posts.head()
impr_posts.shape


(2007080, 12)

## Step 2  Define evaluation metrics

To compare policies I compute:
1. Overall CTR
2. Share of impressions that go to truly stolen posts
3. Share of clicks that go to truly stolen posts
4. Average impressions per original creator

I can then see the trade off between user experience, creator harm, and engagement.


In [25]:
def policy_metrics(df, name):
    # df is an impression level DataFrame after applying a policy
    total_impr = len(df)
    total_clicks = df["clicked"].sum()

    stolen_impr_share = df["is_stolen"].mean()
    # avoid divide by zero for clicks
    stolen_click_share = (
        df.loc[df["is_stolen"], "clicked"].sum() / total_clicks
        if total_clicks > 0 else 0.0
    )

    ctr = total_clicks / total_impr if total_impr > 0 else 0.0

    # creator exposure: how many impressions do non stolen creators get
    creator_exposure = (
        df.loc[~df["is_stolen"]]
        .groupby("author_id")["impression_id"]
        .count()
        .mean()
    )

    return {
        "policy": name,
        "impressions": int(total_impr),
        "ctr": ctr,
        "stolen_impr_share": stolen_impr_share,
        "stolen_click_share": stolen_click_share,
        "avg_impr_per_original_creator": creator_exposure
    }


## Step 3  Baseline performance

First I look at the current feed as is.  
This is my baseline to compare all policies against.


In [26]:
baseline_stats = policy_metrics(impr_posts, "baseline")
baseline_stats


{'policy': 'baseline',
 'impressions': 2007080,
 'ctr': np.float64(0.14959891982382367),
 'stolen_impr_share': np.float64(0.33573499810670226),
 'stolen_click_share': np.float64(0.33625194416782955),
 'avg_impr_per_original_creator': np.float64(911.9240766073872)}

## Step 4  Policy A  Hide all posts predicted as stolen

Here I simulate a strict policy where every impression whose post is
flagged by the detector (`pred_is_stolen = True`) is removed from the feed.

This greatly reduces exposure of stolen content but may also hide some false positives.


In [27]:
policyA_df = impr_posts.loc[~impr_posts["pred_is_stolen"]].copy()
policyA_stats = policy_metrics(policyA_df, "hide_predicted_stolen")
policyA_stats

{'policy': 'hide_predicted_stolen',
 'impressions': 1274403,
 'ctr': np.float64(0.1496363395252522),
 'stolen_impr_share': np.float64(0.35549743683905327),
 'stolen_click_share': np.float64(0.35582625840993826),
 'avg_impr_per_original_creator': np.float64(810.8153998025666)}

## Step 5  Policy B  Downrank predicted stolen posts

Instead of fully hiding flagged content, I also simulate a softer policy.

For impressions where `pred_is_stolen = True`, I randomly drop 50 percent of them.
This mimics downranking  stolen content so it shows up less often but is not completely removed.


In [28]:
np.random.seed(42)

mask_stolen_pred = impr_posts["pred_is_stolen"]

keep_stolen_sample = (
    impr_posts.loc[mask_stolen_pred]
    .sample(frac=0.5, random_state=42)
    .index
)

policyB_df = pd.concat([
    impr_posts.loc[~mask_stolen_pred],
    impr_posts.loc[keep_stolen_sample]
])

policyB_stats = policy_metrics(policyB_df, "downrank_predicted_stolen")
policyB_stats


{'policy': 'downrank_predicted_stolen',
 'impressions': 1640741,
 'ctr': np.float64(0.1495866806522175),
 'stolen_impr_share': np.float64(0.34345579223046174),
 'stolen_click_share': np.float64(0.34419576829521703),
 'avg_impr_per_original_creator': np.float64(736.811901504788)}

## Step 6  Compare policies

Finally I put all policies into one table to see the trade offs between
engagement (CTR) and harm from stolen posts.


In [29]:
results = pd.DataFrame([
    baseline_stats,
    policyA_stats,
    policyB_stats
])

results


,policy,impressions,ctr,stolen_impr_share,stolen_click_share,avg_impr_per_original_creator
0,baseline,2007080,0.149599,0.335735,0.336252,911.924077
1,hide_predicted_stolen,1274403,0.149636,0.355497,0.355826,810.815400
2,downrank_predicted_stolen,1640741,0.149587,0.343456,0.344196,736.811902


In [30]:
seg_country = (
    policyA_df
    .groupby("country")
    .apply(lambda df: pd.Series({
        "impressions": len(df),
        "stolen_impr_share": df["is_stolen"].mean(),
        "ctr": df["clicked"].mean()
    }))
    .sort_values("impressions", ascending=False)
    .head(10)
)

seg_country


/var/folders/kx/5c3b5w651_ngzv1nq36smyxc0000gn/T/ipykernel_9283/2126550886.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda df: pd.Series({


,impressions,stolen_impr_share,ctr
country,,,
US,470607.0,0.362710,0.150068
IN,316497.0,0.371387,0.149341
BR,196184.0,0.313262,0.150145
GB,182571.0,0.384902,0.149202
CA,108544.0,0.304770,0.148438
